In [1]:
#code block 0 start
#import all modules and libraries needed 
!pip install -q scikit-learn statsmodels seaborn


import pandas as pd  # import csv as df (dataframe)
import glob
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

#importing ends here

#show all columns without truncation
pd.set_option('display.max_columns', None)       
#show all rows without truncation
pd.set_option('display.max_rows', None)
#show all columns without getting clipped
pd.set_option('display.max_colwidth', None)      
# Code block 0 end


In [2]:
#code block 1 start
#below are functions used to map codes as defined in the variable library 

def map_occupation(occupation_code):
    if occupation_code in range(11, 15):
        return 'Managers'
    elif occupation_code in range(21, 27):
        return 'Professionals'
    elif occupation_code in range(31, 36):
        return 'Technicians and Associate Professionals'
    elif occupation_code in range(41, 45):
        return 'Clerical Support Workers'
    elif occupation_code in range(51, 55):
        return 'Service and Sales Workers'
    elif occupation_code in range(61, 64):
        return 'Skilled Agricultural, Forestry and Fishery Workers'
    elif occupation_code in range(71, 76):
        return 'Craft and Related Trades Workers'
    elif occupation_code in range(81, 84):
        return 'Plant and Machine Operators and Assemblers'
    elif occupation_code in range(91, 97):
        return 'Elementary Occupations'
    elif occupation_code in range(1, 4):
        return 'Armed Forces Occupations'
    else:
        return 'Unknown'

    
def map_region(region):
    if region == 13:
        return "NCR"
    else:
        return "Outside NCR"

def map_education(grade):
    if grade in [1000, 2000]:
        return "No grade completed"
    elif grade in [10010, 10002]:
        return "Elementary Undergraduate"
    elif grade == 10020:
        return "Elementary Graduate"
    elif grade in [24010, 24002]:
        return "Junior High Undergraduate"
    elif grade == 24020:
        return "Junior High Completed"
    elif grade in [34011, 34021, 34031, 35001]:
        return "Senior High Undergraduate"
    elif grade in [34012, 34022, 34032, 35002]:
        return "Senior High Graduate"
    elif grade in [40010, 50010]:
        return "Post Secondary Undergraduate"
    elif 40011 <= grade <= 49999 or 50011 <= grade <= 59999:
        return "Post Secondary Graduate"
    elif grade == 60010:
        return "College Undergraduate"
    elif grade == 60000 or (60011 <= grade <= 69999) or (70000 <= grade <= 79999) or (80000 <= grade <= 89999):
        return "College Graduate"
    else:
        return "Other"




def classify_sector(code):
    try:
        code = int(code)
        if 1 <= code <= 3:
            return 'AGRICULTURE'
        elif 5 <= code <= 43:
            return 'INDUSTRY'
        elif 45 <= code <= 99:
            return 'SERVICES'
        else:
            return 'OTHER'
    except:
        return 'UNKNOWN'

#functions for mapping ends here
#code 1 block end


In [3]:
#code 2 block start
#the code for combining all the data in the csv begins here
#load and combine all csv from microdata folder
files = glob.glob(r"C:\Users\iamt\Documents\UP Notes\3rd Year - 2nd Sem\Econ 138 WZQ\econ_trev-20250512T062813Z-1-001\econ_trev\microdata_csv"
) #path of folder/dir containing csv files
df_list = []

#what we are doing in this loop here is basically not including the dataset that has no wage/basic pay data
for file in files:
    temp_df = pd.read_csv(file, low_memory=False)
    if 'PUFC25_PBASIC' in temp_df.columns:
        df_list.append(temp_df)
    else:
        print(f"Skipping file (missing PUFC25_PBASIC): {file}")

        
data = pd.concat(df_list,ignore_index=True)

#code below just chooses relevant column that is needed, other columns are discarded since the dataframe is too large
cols = ['PUFSVYMO', 'PUFSVYYR', 'PUFC05_AGE','PUFC11_WORK', 'PUFC12_JOB', 'PUFC14_PROCC', 'PUFC16_PKB', 'PUFC04_SEX', 'PUFNEWEMPSTAT','PUFC27_NJOBS',
         'PUFC27_NJOBS', 'PUFC28_THOURS', 'PUFC18_PNWHRS', 'PUFC19_PHOURS','PUFC25_PBASIC', 'PUFC07_GRADE','PUFREG'] #relevant columns
data = data[cols] #remove irrelevant columns
data = data.loc[:, ~data.columns.duplicated()]


# Define age bins and labels (we make a new column here, age group to group it better for clarity and presentation)
bins = [15, 24, 34, 44, 54, 64, 100]
labels = ['15-24', '25-34', '35-44', '45-54', '55-64', '65+']
# Create new age group column
data['age group'] = pd.cut(data['PUFC05_AGE'], bins=bins, labels=labels, right=True)

#data['PUFC14_PROCC'] = pd.to_numeric(data['PUFC14_PROCC'].astype(str).str.strip(), errors='coerce')
data['PUFREG'] = pd.to_numeric(data['PUFREG'].astype(str).str.strip(), errors='coerce')
data['PUFC07_GRADE'] = pd.to_numeric(data['PUFC07_GRADE'].astype(str).str.strip(), errors='coerce')
data['PUFC25_PBASIC'] = pd.to_numeric(data['PUFC25_PBASIC'].astype(str).str.strip(), errors='coerce')
data = data.dropna(subset=['PUFC25_PBASIC']) #drop fields with empty data
data['PUFC28_THOURS'] = pd.to_numeric(data['PUFC28_THOURS'].astype(str).str.strip(), errors='coerce')
data = data.dropna(subset=['PUFC28_THOURS']) #drop fields with empty data
data['PUFSVYMO'] = data['PUFSVYMO'].astype(str).str.zfill(2) #to ensure that all fields have 2 digits for motnhs
data['PUFSVYYR'] = data['PUFSVYYR'].astype(int) #convert data to int
data['PUFSVYYR'] = data['PUFSVYYR'].apply(lambda x: x + 2000 if x < 100 else x) #year in 2019 is written as 19 instead of 2019, which messed up data grouping
data['date'] = pd.to_datetime(data['PUFSVYYR'].astype(str) + '-' + data['PUFSVYMO'] + '-01')
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['education_level'] = data["PUFC07_GRADE"].apply(map_education)
data['region'] = data["PUFREG"].apply(map_region)
#data['occupation'] = data["PUFC14_PROCC"].apply(map_occupation)
#code 2 block end



ValueError: No objects to concatenate

In [8]:
#code 3 blk start
#we group and aggregate data below to prepare it for k-means clustering
grouped = data.groupby(['date', 'PUFC16_PKB', 'PUFC04_SEX', 'age group','education_level','region', 'year']).agg({'PUFC28_THOURS' : 'mean', 'PUFC25_PBASIC' : 'median'}).reset_index()
grouped = grouped.dropna(subset=['PUFC28_THOURS'])
grouped = grouped.dropna(subset=['PUFC25_PBASIC'])
grouped['sector'] = grouped['PUFC16_PKB'].apply(classify_sector)




grouped.rename(columns={'PUFC28_THOURS' : 'avg_hours_worked'}, inplace=True)
grouped.rename(columns={'PUFC25_PBASIC' : 'avg_wage'}, inplace=True)

avg_wage_by_year = grouped.groupby('year')['avg_wage'].mean().reset_index()


national_avg_hours = grouped.groupby('year')['avg_hours_worked'].transform('mean')
grouped['deviation_from_natl_hours'] = grouped['avg_hours_worked'] - national_avg_hours
grouped['centered_deviation'] = grouped['deviation_from_natl_hours'] - grouped['deviation_from_natl_hours'].mean()
#code 3 blk end


NameError: name 'data' is not defined

In [9]:
#code 4 blk start

sector_dummies = pd.get_dummies(grouped['sector'], prefix='sector')

X = pd.concat([
    sector_dummies,
    pd.get_dummies(grouped['year'],prefix='year'),
    pd.get_dummies(grouped['region'],prefix='region'),
    pd.get_dummies(grouped['education_level'],prefix='education_level'),
    pd.get_dummies(grouped['PUFC04_SEX'], prefix='sex'),
    pd.get_dummies(grouped['age group'], prefix='age'),
    grouped[['avg_hours_worked','avg_wage']]
], axis=1)



##label_sex = LabelEncoder()
##label_age = LabelEncoder()
##label_industry = LabelEncoder()
##
##grouped['sex_encoded'] = label_sex.fit_transform(grouped['PUFC04_SEX'])
##grouped['age_encoded'] = label_age.fit_transform(grouped['age group'])
##grouped['industry_encoded'] = label_industry.fit_transform(grouped['PUFC16_PKB'])

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit KMeans
kmeans = KMeans(n_clusters=3, random_state=42) #we choose 3 clusters for simplicity, and random_state = 42 so that results will not vary that much per run
grouped['cluster'] = kmeans.fit_predict(X_scaled)


#occupation_mode = grouped.groupby('cluster')['occupation'].agg(lambda x: x.mode()[0])
sector_mode = grouped.groupby('cluster')['sector'].agg(lambda x: x.mode()[0])
region_mode = grouped.groupby('cluster')['region'].agg(lambda x: x.mode()[0])
sex_mode = grouped.groupby('cluster')['PUFC04_SEX'].agg(lambda x: x.mode()[0])
educ_mode = grouped.groupby('cluster')['education_level'].agg(lambda x: x.mode()[0])
age_mode = grouped.groupby('cluster')['age group'].agg(lambda x: x.mode()[0])
avg_hours = grouped.groupby('cluster')['avg_hours_worked'].mean()
avg_wage = grouped.groupby('cluster')['avg_wage'].mean()

cluster_summary = pd.DataFrame({
    'Most Common Sector': sector_mode,
    'Most Common Region': region_mode,
    'Most Common Sex': sex_mode,
    'Most Common Educ Level' : educ_mode,
    'Most Common Age Group': age_mode,
    'Average Working Hours': avg_hours.round(2),
    'Average Wage': avg_wage.round(2)
})

print(cluster_summary) #we print cluster_summary to provide a tableview form for analysis
#code 4 block end


NameError: name 'grouped' is not defined

In [12]:
#code 5 blk start
#code below shows linear regression

#the first step for the linear regression would be to choose the target (average daily wage) and features
features = ['sector', 'PUFC04_SEX', 'age group', 'education_level', 'region', 'year'] #sex 01 - male, 02- female ; year (2019-2023)
A = pd.get_dummies(grouped[features], drop_first=True)
A['deviation_from_natl_hours'] = grouped['centered_deviation']


if 'sector_SERVICES' in A.columns:
    A['services_x_deviation'] = A['sector_SERVICES'] * A['deviation_from_natl_hours']
if 'sector_INDUSTRY' in A.columns:
    A['industry_x_deviation'] = A['sector_INDUSTRY'] * A['deviation_from_natl_hours']

for col in A.columns:
    if col.startswith("year_"):
        A[f'{col}_x_deviation'] = A[col] * A['deviation_from_natl_hours']

A = sm.add_constant(A)

B = grouped['avg_wage']

# code below just ensures that everything is int-type (since importing the values from the dataset usually results to string-type)
A = A.apply(pd.to_numeric, errors='coerce')
B = pd.to_numeric(B, errors='coerce')

#we drop rows with empty values
valid = A.notnull().all(axis=1) & B.notnull()
A = A[valid]
B = B[valid]

#convert bool-type to int-type 
A = A.astype({col: 'int' for col in A.select_dtypes(bool).columns})

#we check here if all the variables are valid
print("=== A.dtypes ===")
print(A.dtypes)
print("\n=== B.dtype ===")
print(B.dtype)
print("\nAny object dtype in A:", A.dtypes[A.dtypes == 'object'])
print("\nAny NaNs in A?", A.isnull().any().any())
print("Any NaNs in B?", B.isnull().any())

#fit the regression model
model = sm.OLS(B, A).fit()

#show summary
print(model.summary())

#code 5 block end


NameError: name 'grouped' is not defined

In [13]:
#code 6 blk start
grouped = grouped.loc[A.index]  # align with regression data
grouped['predicted_wage'] = model.predict(A) # preditction 
#code 6 blk end


NameError: name 'grouped' is not defined

In [14]:
#visualization of data begins here

#paste the code block per number here, basically from the comment up to plt.show()

# 1. Cluster-wise distribution of average working hours
plt.figure(figsize=(8, 5))
sns.boxplot(x='cluster', y='avg_hours_worked', data=grouped)
plt.title('Cluster-wise Distribution of Average Working Hours')
plt.xlabel('Cluster')
plt.ylabel('Average Working Hours')
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [15]:
# 2. Cluster-wise distribution of average wage
plt.figure(figsize=(8, 5))
sns.boxplot(x='cluster', y='avg_wage', data=grouped)
plt.title('Cluster-wise Distribution of Average Wage')
plt.xlabel('Cluster')
plt.ylabel('Average Wage')
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [16]:
# 3. Scatter plot of wage vs. hours colored by cluster
plt.figure(figsize=(8, 6))
sns.scatterplot(data=grouped, x='avg_hours_worked', y='avg_wage', hue='cluster', palette='Set2', s=80)
plt.title('Clusters Based on Wage and Working Hours')
plt.xlabel('Average Hours Worked')
plt.ylabel('Average Wage')
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [17]:
# 4. Multivariate pairplot
merged = grouped.copy()
merged['cluster'] = grouped['cluster'].astype(str)
sns.pairplot(merged[['avg_hours_worked', 'avg_wage', 'cluster']], hue='cluster', palette='Set2')
plt.show()



NameError: name 'grouped' is not defined

In [18]:
#1. Trend of average wages over time
plt.figure(figsize=(10, 6))
sns.lineplot(data=avg_wage_by_year, x='year', y='avg_wage', marker='o')
plt.title('Average Wage Over Time (2019-2023)', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Average Wage', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [19]:
# 2. Predicted average wage by education level
plt.figure(figsize=(10, 6))
sns.barplot(data=grouped, x='education_level', y='predicted_wage', errorbar=None)
plt.xticks(rotation=45)
plt.title("Predicted Average Daily Wage by Education Level")
plt.ylabel("Predicted Wage")
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [20]:
# 3. Predicted wage over time
plt.figure(figsize=(10, 6))
sns.lineplot(data=grouped, x='year', y='predicted_wage', marker='o')
plt.title("Predicted Wage Over Time")
plt.xlabel("Year")
plt.ylabel("Predicted Wage")
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [21]:
# 4. Predicted wage by region
plt.figure(figsize=(12, 6))
sns.boxplot(data=grouped, x='region', y='predicted_wage')
plt.title("Predicted Wage by Region")
plt.xticks(rotation=45)
plt.ylabel("Predicted Wage")
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined